In [1]:
# import packages

In [2]:
from pandas.io.json import json_normalize
import json
from pandasql import sqldf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import io
import requests
import json
import base64
import datetime
from urllib.parse import urlencode

In [3]:
# client id

In [4]:
client_id = '6e24789ec7874259a220f747fcd7e93e'
client_secret = '9bd8ff1ad2f34ef6a87f00124be070b2'

In [5]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"



    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def getClientCreds(self):
        '''Returns b64 encoded string'''
        client_id = self.client_id
        client_secret = self.client_secret

        if client_id == None or client_secret == None:
            raise Exception('Must set a client id and secret')

        client_creds  = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()

    def getTokenHeader(self):
        client_creds_b64 = self.getClientCreds()
        return {
            'Authorization':f"Basic {client_creds_b64}"
        }

    def getTokenData(self):
        return {
            "grant_type":"client_credentials"
        }

    def perform_auth(self):
        token_url = self.token_url
        token_data = self.getTokenData()
        token_header = self.getTokenHeader()

        r = requests.post(token_url, data=token_data, headers=token_header)

        if r.status_code  not in range(200,299):
            raise Exception('Could not authenticate client.')
            #return False

        now = datetime.datetime.now()
        token_response_data = r.json()
        access_token = token_response_data['access_token']
        expires_in = token_response_data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now


        return True

    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now= datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}", 
        }
        return headers
    
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f'https://api.spotify.com/{version}/{resource_type}/{lookup_id}'
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers = headers)
        if r.status_code not in range (200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id ):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id ):
        return self.get_resource(_id, resource_type='artists')
    
    def get_playlist(self, _id):
        return self.get_resource(_id, resource_type='playlists')
    
    
    def base_search(self, query_params): #type
        access_token = self.get_access_token()
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_URL = f"{endpoint}?{query_params}"
        r = requests.get(lookup_URL, headers=headers)
        if r.status_code not in range (200, 299):
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist'):
        if query == None:
            raise Exception('A query is required')
        if isinstance(query, dict):
            query = ' '.join([f'{k}:{v}' for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == 'or' or operator.lower() == 'not':
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f'{query} {operator} {operator_query}'
        query_params = urlencode({'q': query, 'type': search_type.lower()})
        return self.base_search(query_params)

In [6]:
spotify = SpotifyAPI(client_id, client_secret)

In [14]:
# Blinding Lights

In [8]:
Blinding_Lights = spotify.search({'track': 'Blinding Lights', 'artist': 'The'}, search_type ='track')
Blinding_Lights

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3ABlinding+Lights+artist%3AThe&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ'},
       'href': 'https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ',
       'id': '1Xyo4u8uXC1ZmMpatF05PJ',
       'name': 'The Weeknd',
       'type': 'artist',
       'uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
    

In [9]:
test = pd.DataFrame.from_dict(Blinding_Lights,orient='columns')
test

,tracks
href,https://api.spotify.com/v1/search?query=track%...
items,"[{'album': {'album_type': 'album', 'artists': ..."
limit,20
next,https://api.spotify.com/v1/search?query=track%...
offset,0
previous,None
total,196


In [10]:
test= test.tracks.apply(pd.Series)
test

/var/folders/91/wchv5p25455f2k8078kdjvm40000gn/T/ipykernel_28078/2977931382.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  test= test.tracks.apply(pd.Series)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
href,https://api.spotify.com/v1/search?query=track%...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
items,"{'album': {'album_type': 'album', 'artists': [...","{'album': {'album_type': 'compilation', 'artis...","{'album': {'album_type': 'compilation', 'artis...","{'album': {'album_type': 'compilation', 'artis...","{'album': {'album_type': 'compilation', 'artis...","{'album': {'album_type': 'compilation', 'artis...","{'album': {'album_type': 'compilation', 'artis...","{'album': {'album_type': 'compilation', 'artis...","{'album': {'album_type': 'compilation', 'artis...","{'album': {'album_type': 'compilation', 'artis...","{'album': {'album_type': 'compilation', 'artis...","{'album': {'album_type': 'compilation', 'artis...","{'album': {'album_type': 'compilation', 'artis...","{'album': {'album_type': 'compilation', 'artis...","{'album': {'album_type': 'single', 'artists': ...","{'album': {'album_type': 'single', 'artists': ...","{'album': {'album_type': 'album', 'artists': [...","{'album': {'album_type': 'compilation', 'artis...","{'album': {'album_type': 'album', 'artists': [...","{'album': {'album_type': 'compilation', 'artis..."
limit,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
next,https://api.spotify.com/v1/search?query=track%...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
offset,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
previous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
total,196.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
test2 = pd.DataFrame(test.iloc[1])
test2

,items
0,"{'album': {'album_type': 'album', 'artists': [..."
1,"{'album': {'album_type': 'compilation', 'artis..."
2,"{'album': {'album_type': 'compilation', 'artis..."
3,"{'album': {'album_type': 'compilation', 'artis..."
4,"{'album': {'album_type': 'compilation', 'artis..."
5,"{'album': {'album_type': 'compilation', 'artis..."
6,"{'album': {'album_type': 'compilation', 'artis..."
7,"{'album': {'album_type': 'compilation', 'artis..."
8,"{'album': {'album_type': 'compilation', 'artis..."
9,"{'album': {'album_type': 'compilation', 'artis..."


In [12]:
test3 = test2['items'].apply(pd.Series)
test3

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,200040,False,{'isrc': 'USUG11904206'},{'spotify': 'https://open.spotify.com/track/0V...,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,0VjIjW4GlUZAMYd2vXMi3b,False,Blinding Lights,90,https://p.scdn.co/mp3-preview/579494c4709a8cc9...,9,track,spotify:track:0VjIjW4GlUZAMYd2vXMi3b
1,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AE, AL, AR, AT, BA, BE, BG, BH, BO, BR, CA, C...",1,201573,False,{'isrc': 'USUG11904206'},{'spotify': 'https://open.spotify.com/track/6w...,https://api.spotify.com/v1/tracks/6w7zPSW5zvAs...,6w7zPSW5zvAsMpU0CKuaN0,False,Blinding Lights,0,https://p.scdn.co/mp3-preview/c4c0d1968268606a...,54,track,spotify:track:6w7zPSW5zvAsMpU0CKuaN0
2,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, BR, C...",1,201573,False,{'isrc': 'USUG11904206'},{'spotify': 'https://open.spotify.com/track/4m...,https://api.spotify.com/v1/tracks/4mWcNMhTh3a1...,4mWcNMhTh3a13x6e5t74cO,False,Blinding Lights,0,https://p.scdn.co/mp3-preview/c4c0d1968268606a...,19,track,spotify:track:4mWcNMhTh3a13x6e5t74cO
3,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AE, AL, AR, AT, AU, BA, BG, BH, BO, BR, CA, C...",1,201573,False,{'isrc': 'USUG11904206'},{'spotify': 'https://open.spotify.com/track/4F...,https://api.spotify.com/v1/tracks/4FqwXdBZosVW...,4FqwXdBZosVWE2E80TQ8Us,False,Blinding Lights,0,https://p.scdn.co/mp3-preview/c4c0d1968268606a...,8,track,spotify:track:4FqwXdBZosVWE2E80TQ8Us
4,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, BR, C...",1,201573,False,{'isrc': 'USUG11904206'},{'spotify': 'https://open.spotify.com/track/1J...,https://api.spotify.com/v1/tracks/1JVsdYtWDPAS...,1JVsdYtWDPASUHRiBwIqVC,False,Blinding Lights,0,https://p.scdn.co/mp3-preview/c4c0d1968268606a...,18,track,spotify:track:1JVsdYtWDPASUHRiBwIqVC
5,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, BR, C...",1,201573,False,{'isrc': 'USUG11904206'},{'spotify': 'https://open.spotify.com/track/0k...,https://api.spotify.com/v1/tracks/0k0HVjt1hUSB...,0k0HVjt1hUSB1i74oO1l0Z,False,Blinding Lights,0,https://p.scdn.co/mp3-preview/c4c0d1968268606a...,3,track,spotify:track:0k0HVjt1hUSB1i74oO1l0Z
6,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, BR, C...",1,201573,False,{'isrc': 'USUG11904206'},{'spotify': 'https://open.spotify.com/track/1R...,https://api.spotify.com/v1/tracks/1RRWwBdmToBw...,1RRWwBdmToBw8fkc10Z7RP,False,Blinding Lights,1,https://p.scdn.co/mp3-preview/c4c0d1968268606a...,3,track,spotify:track:1RRWwBdmToBw8fkc10Z7RP
7,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, BR, C...",1,201573,False,{'isrc': 'USUG11904206'},{'spotify': 'https://open.spotify.com/track/1c...,https://api.spotify.com/v1/tracks/1cI65eHo2StP...,1cI65eHo2StP51vJgQCedO,False,Blinding Lights,0,https://p.scdn.co/mp3-preview/c4c0d1968268606a...,9,track,spotify:track:1cI65eHo2StP51vJgQCedO
8,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, BR, C...",1,201573,False,{'isrc': 'USUG11904206'},{'spotify': 'https://open.spotify.com/track/4K...,https://api.spotify.com/v1/tracks/4KUnfcxR8l49...,4KUnfcxR8l49BQkeWvpd5Z,False,Blindin

In [13]:
test3.head(1)

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,200040,False,{'isrc': 'USUG11904206'},{'spotify': 'https://open.spotify.com/track/0V...,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,0VjIjW4GlUZAMYd2vXMi3b,False,Blinding Lights,90,https://p.scdn.co/mp3-preview/579494c4709a8cc9...,9,track,spotify:track:0VjIjW4GlUZAMYd2vXMi3b
